## Article Supervisor Agent
In the previous notebooks, we demonstrated 2 agents where the first agent is designed to write a article for an upcoming release of a new show/movie. The second agent is created to review and critiques the output from the first agent, and give suggestions to improve the quality based on the specific guidelines. In this notebook, we'll focus on connecting these agents together so that the process of writing and reviewing process could be performed iteratively to build the highest quality PR article. 

We'll be using the multi-agent collaborator feature on Amazon Bedrock to implement the solution.

[Multi-agent Collaboration](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-multi-agents-collaboration.html) is a Amazon Bedrock Agents native capability that enables a hierarchical collaboration between agents. You can now enable agent collaboration and associate secondary agents to a supervisor one. These secondary agents can be any existing agent within the same account, including agents that have collaboration themselves. 

In this notebook, we will create a supervisor agent that interacts with the PR Writer agent and PR reviewer agent created in the previous notebooks.

The following architecture highlights the supervisor agent in the multi-agent collaborator environment:

<img src="../imgs/pr-reflection-agent-supervisor.png" width="600">

Let's get started!

First we restore the variables from the previous notebook

In [ ]:
%store -r

## Create a Article Supervisor agent using Amazon Bedrock Agent
On this section we declare global variables that will be act as helpers during entire notebook.

In [ ]:
import boto3
import os
import json
import time
import uuid
from datetime import datetime

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'us.amazon.nova-pro-v1:0'
]

## Importing helper functions
On following section, we're adding bedrock_agent_helper.py and knowledge_base_helper on Python path, so the files can be recognized and their functionalities can be invoked.

In general, the helper functions handle common tasks including agent creation, knowledge bases for bedrock creation and accessing data on S3.

In [4]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)

agents = AgentsForAmazonBedrock()

## Article Supervisor Agent
Create a supervisor agent to connect the Artcle Generation and Reviewer agents to iteratively improve the quality of the article. 

We will provide the following instructions to our agent:
```
You are a supervisor AI agent. You are given 2 agents capable of following tasks:

1. PR Writer agent - Writes a draft of PR article based on the given instructions and/or recommendations.

2. PR Reviewer agent - Reviews the PR article generated by the PR Writer agent to provide recommendation to improve the PR article.

You are to leverage the above agents to perform PR article generation and reviews collaboratively and iteratively to arrive at the best version of the PR article. 

Here are the guidelines:
- You should only iterate the draft/review iterations for maximum of 3 time before returning the final version of the PR article to the user. 

The final draft of the PR must be a complete version formatted in markdown in the final response.
```

In [ ]:
article_supervisor_agent_name = "article_supervisor_agent"
supervisor_agent = agents.create_agent(
    article_supervisor_agent_name,
    """You are a supervisor AI agent that creates a high quality PR articles based on user input. 
""",
    """You are a supervisor AI agent. You are given 2 agents capable of following tasks:

1. Article Generation agent - Writes a draft of article based on the given instructions and/or recommendations.

2. Article Reviewer agent - Reviews the article generated by the Article Generation agent to provide recommendation to improve the article.

You are to leverage both the agents to work collaboratively and iteratively to arrive at the best version of the article. 

Here are the guidelines:
- You should only iterate the draft and review iterations for maximum of 3 time before returning the final version of the article to the user. 

The final draft of the article must be a complete version formatted in markdown in the final response. Do not provide any explanation in the final response, return only the final article in markdown format.
""",
    agent_foundation_model,
    agent_collaboration='SUPERVISOR'
)

In [ ]:
article_supervisor_agent_id = supervisor_agent[0]

### Associate Collaborators

On this section, we're going to recover alias_id from previous agents (sub-agents) to add all of them inside energy one (which is multi-agent collaborator).

In [ ]:
sub_agents_list = [
    {
        'sub_agent_alias_arn': article_generation_agent_alias_arn,
        'sub_agent_instruction': """Delegate article writing task to the Article Generation Agent, ensuring adherence to its specific protocols and capabilities.""",
        'sub_agent_association_name': 'ArticleGenerationAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    },
    {
        'sub_agent_alias_arn': article_reviewer_agent_alias_arn,
        'sub_agent_instruction': """Delegate article review task to the Article Reviewer Agent, ensuring most helpful feedback is given by the agent.""",
        'sub_agent_association_name': 'ArticleReviewerAgent',
        'relay_conversation_history': 'TO_COLLABORATOR'
    }
]

Associate all sub-agents with this one (main one).

In [ ]:
article_supervisor_agent_alias_id, article_supervisor_agent_alias_arn = agents.associate_sub_agents(
    article_supervisor_agent_id, sub_agents_list
)

## Invoking multi-agent Collaborator

With the supervisor agent creation, we can now put everything together by invoking the supervisor agent with PR article creation queries to build a high quality PRs. 

We can now set a dictionary with agents names for better visualization of the traces

In [ ]:
multi_agent_names = {
    f"{article_generation_agent_id}/{article_generation_alias_id}": article_generation_agent_name,
    f"{article_reviewer_agent_id}/{article_reviewer_agent_alias_id}": article_reviewer_agent_name,
    f"{article_supervisor_agent_id}/{article_supervisor_agent_alias_id}": article_supervisor_agent_name
}

Let's test it out by providing a prompt to the supervisor agent to create the best quality PR article for an upcoming movie.

In [39]:
pr_query_prompt = """Write a PR news for an upcoming movie given the following information: 
Title: Midnight Vendetta
Tagline: “When the clock strikes twelve, justice wears no mask.”
Genre: Action/Thriller
Release Date: November 22, 2025 (Theatrical & IMAX)

Logline:
A disgraced former MI6 agent infiltrates a glittering Dubai masquerade ball to dismantle a trillion-dollar cyberweapons syndicate—but must confront his deadliest enemy: the traitor who framed him for murder.

Key Production/Cast Details:
Director: David Leitch (Atomic Blonde, Bullet Train)—promises “brutally elegant fight choreography blending Bourne-style close combat with Dubai’s opulent settings.”
Studio: Pika Pictures (Budget: $200M)
Filming Locations: Dubai’s Burj Khalifa, Palm Jumeirah, and a custom-built 360-degree rotating ballroom set for the climactic fight.
Soundtrack: Pulse-pounding score by Ludwig Göransson 
"""

In [ ]:
%%time
session_id:str = str(uuid.uuid1())


article_supervisor_agent_response = agents.invoke(
    pr_query_prompt,
    article_supervisor_agent_id,
    session_id=session_id,
    enable_trace=True,
    trace_level = "all",
    multi_agent_names=multi_agent_names
)

In [ ]:
print(article_supervisor_agent_response)

# Summary
In this notebook, we demonstrated how you could use Bedrock Multiagent Collaboration to build a PR article agent using the reflection pattern. First, we built a PR writer agent responsible for curating a PR article by giving it access to a knowledge base that contains a repository of prestine articles as reference. Next, we built a PR reviewer agent that performs review and suggestions to improve the quality of the PR article. Finally, we built a supervisor agent that leverages the collaboration between these 2 agents to improve the quality of the PR article iteratively. The final PR article is returned to the user. 